In [10]:
import geopandas as gpd
import matplotlib.pyplot as plt
import time

In [2]:
# Read the two shapefiles
village = gpd.read_file(r"D:\MPSeDC\Admin_boundary_Ver.17_GCS\Village.shp")
whater_Body = gpd.read_file(r"D:\MPSeDC\SIPRI\Watershed_Prioritization_Criteria\Watershed\Watershed\Watershed.shp")

# Check CRS (coordinate reference system)
print("village CRS:", village.crs)
print("whater_Body  CRS:", whater_Body.crs)

# If CRS are different, reproject one to match the other
if village.crs != whater_Body.crs:
    whater_Body = whater_Body.to_crs(village.crs)

# # Plot both shapefiles together
# fig, ax = plt.subplots(figsize=(8, 8))
# village.plot(ax=ax, color="lightblue", edgecolor="black", alpha=0.5)
# whater_Body.plot(ax=ax, color="orange", edgecolor="red", alpha=0.5)

# plt.title("Two Polygon Shapefiles")
# plt.show()


village CRS: EPSG:4326
whater_Body  CRS: EPSG:4326


In [4]:
print(village.columns)

Index(['code', 'class', 'sub_class', 'vil_nm_h', 'vil_nm_e', 'lgd_gp_nm',
       'teh_nm', 'b_nm', 'dist_nm', 'div_nm', 'ac_nm', 'pc_nm', 'state_nm',
       'b_cd', 'dist_cd', 'div_cd', 'ac_cd', 'pc_cd', 'state_cd', 'area_hecta',
       'lgd_gp_cd', 'teh_cd', 'vil_cd', 'bhu_cd', 'ccode_11', 'lgdcd', 'FID_1',
       'geometry'],
      dtype='object')
Index(['AREA', 'PERIMETER', 'NVDAWSHED_', 'NVDAWSHED1', 'REP_CODE', 'Basin',
       'Watershed', 'neemuch_15', 'FID_1', 'geometry'],
      dtype='object')


In [18]:

# Intersect waterbodies with villages
intersection = gpd.overlay(whater_Body, village, how="intersection", keep_geom_type=False)
print(intersection)

# Count waterbody pieces per village using FID_1
counts = intersection.groupby("FID_1_1").size().reset_index(name="waterbody_count")
counts

            AREA     PERIMETER  NVDAWSHED_  NVDAWSHED1 REP_CODE Basin  \
0       0.001244  15320.621791           2         220  AGR1335    2D   
1       0.000839  14856.958529           3         219  AGR1335    2D   
2       0.001065  15395.269706           4         221  AGR1335    2D   
3       0.000792  15987.527040           5         218  AGR1335    2D   
4       0.000792  15987.527040           5         218  AGR1335    2D   
...          ...           ...         ...         ...      ...   ...   
220425  0.001273  22356.252792       39619          20   AGR706    5C   
220426  0.001526  22245.875767       39620          25   AGR706    5C   
220427  0.001243  20067.750308       39623          22   AGR706    5C   
220428  0.001172  15661.090835       39626          24   AGR706    5C   
220429  0.001150  26184.809056       39629          24   AGR706    5C   

       Watershed  neemuch_15  FID_1_1 code  ... state_cd    area_hecta  \
0          2D1A3         0.0        0   01  ...  

In [24]:
print(intersection.columns)

Index(['AREA', 'PERIMETER', 'NVDAWSHED_', 'NVDAWSHED1', 'REP_CODE', 'Basin',
       'Watershed', 'neemuch_15', 'FID_1_1', 'code', 'class', 'sub_class',
       'vil_nm_h', 'vil_nm_e', 'lgd_gp_nm', 'teh_nm', 'b_nm', 'dist_nm',
       'div_nm', 'ac_nm', 'pc_nm', 'state_nm', 'b_cd', 'dist_cd', 'div_cd',
       'ac_cd', 'pc_cd', 'state_cd', 'area_hecta', 'lgd_gp_cd', 'teh_cd',
       'vil_cd', 'bhu_cd', 'ccode_11', 'lgdcd', 'FID_1_2', 'geometry'],
      dtype='object')


In [26]:
print(intersection.shape)

(220430, 37)


In [34]:
# Count waterbody pieces per village using FID_1
counts = intersection.groupby("FID_1_2").size().reset_index(name="waterbody_count")

rename = counts.rename(columns={
   "FID_1_2":"FID_1"
})
rename

,FID_1,waterbody_count
0,0,19
1,1,1
2,2,2
3,3,3
4,4,1
...,...,...
58693,58695,2
58694,58696,3
58695,58697,3
58696,58698,3


In [41]:

# Merge counts back to villages using village index (assuming FID_1 corresponds to index)
villages = village.merge(rename, how="left")
# print(villages["waterbody_count"])
# Fill NaN (villages with no waterbody)
villages["waterbody_count"] = villages["waterbody_count"].fillna(0).astype(int)

# Save
villages.to_file(r"D:\MPSeDC\SIPRI\Watershed_Prioritization_Criteria\Watershed\villages_with_waterbody_count_updated.shp")

print(villages[["vil_cd", "waterbody_count"]].head())

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_16844\3512025995.py:8: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  villages.to_file(r"D:\MPSeDC\SIPRI\Watershed_Prioritization_Criteria\Watershed\villages_with_waterbody_count_updated.shp")
C:\Users\Lenovo\.conda\envs\Gis_venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'waterbody_count' to 'waterbody_'
  ogr_write(


   vil_cd  waterbody_count
0  453859               19
1  948169                1
2  948200                2
3  948209                3
4  948171                1
